In [ ]:
%run "0_Workspace_setup.ipynb"

from ipywidgets import widgets

from NHM_helpers.NHM_helpers import hrus_by_poi, hrus_by_seg, subset_stream_network
from NHM_helpers.NHM_hydrofabric import make_hf_map_elements

from NHM_helpers.map_template import *
from NHM_helpers.NHM_Assist_utilities import make_plots_par_vals
import webbrowser

In [ ]:
(
    hru_gdf,
    hru_txt,
    hru_cal_level_txt,
    seg_gdf,
    seg_txt,
    nwis_gages_aoi,
    poi_df,
    default_gages_file,
    gages_df,
    gages_txt,
    _,
    HW_basins_gdf,
    HW_basins,
) = make_hf_map_elements(
    NHM_dir,
    model_dir,
    GIS_format,
    param_filename,
    control_file_name,
    nwis_gages_file,
    gages_file,
    nhru_params,
    nhru_nmonths_params,
    nwis_gage_nobs_min,
)

con.print(
    f"{workspace_txt}\n",
    f"\n{gages_txt}{seg_txt}{hru_txt}",
    f"\n     {hru_cal_level_txt}",
)

## Build plots at each POI that show par values of contributing HRUs

In [ ]:
make_plots_par_vals(
    poi_df,
    hru_gdf,
    param_filename,
    nhru_params,
    nhru_nmonths_params,
    Folium_maps_dir,
)

#### &#x270D;<font color='green'>**Enter Information:**</font> **Select a parameter to view below**. 
> These parameters are dimensioned **by hru**, or **by hru and by months**.

In [ ]:
cal_hru_params = nhru_params + nhru_nmonths_params

par_sel = cal_hru_params[4]
# sel_flag = False

v = widgets.Dropdown(
    options=cal_hru_params,
    value=cal_hru_params[0],
    description="Select a parameter to view in plots:",
)


def on_change(change):
    global par_sel, sel_flag
    if change["type"] == "change" and change["name"] == "value":
        par_sel = v.value
        # sel_flag = True


v.observe(on_change)

display(v)

#### &#x1F6D1;If new selections are made above,
><font color='green'>**select this cell**</font>, then select <font color='green'>**Run Selected Cell and All Below**</font> from the Run menu in the toolbar.

#### &#x270D;<font color='green'>**Enter Information:**</font> **Select a month. Default setting is "July"**. 
> 

In [ ]:
mo_num_dict = {
    "January": 1,
    "February": 2,
    "March": 3,
    "April": 4,
    "May": 5,
    "June": 6,
    "July": 7,
    "August": 8,
    "September": 9,
    "October": 10,
    "November": 11,
    "December": 12,
}

mo_names = list(mo_num_dict.keys())

mo_name = "July"  # set default value
mo_sel = mo_num_dict[mo_name]

try:
    pdb.get(par_sel).dimensions["nmonths"].size

except:
    print(f"{par_sel} dimensioned by hru only.")
    mo_sel = None

else:
    print(f"{par_sel} dimensioned by hru and months. Please select a month below.")
    # mo_name = "July"  # set default value
    # mo_sel = mo_num_dict[mo_name]

    m = widgets.Dropdown(
        options=mo_names,
        value=mo_names[6],  # set default value
        description="Select a month to display on the map:",
    )

    def on_change(change):
        global mo_sel, mo_name, mo_num_dict
        if change["type"] == "change" and change["name"] == "value":
            mo_name = m.value
            mo_sel = mo_num_dict[mo_name]

    m.observe(on_change)

    display(m)

#### &#x1F6D1;If new selections are made above,
><font color='green'>**select this cell**</font>, then select <font color='green'>**Run Selected Cell and All Below**</font> from the Run menu in the toolbar.

## Make interactive map of selected parameter values in the NHM domain

In [ ]:
map_file = make_par_map(
    hru_gdf,
    HW_basins_gdf,
    HW_basins,
    poi_df,
    _,
    par_sel,
    mo_sel,
    mo_name,
    nhru_params,
    Folium_maps_dir,
    seg_gdf,
    nwis_gages_aoi,
    gages_df,
    html_maps_dir,
    param_filename,
    subbasin,
)

map_file_str =f"{map_file}"
_ = webbrowser.open(map_file_str, new=2)